# New York City Taxi Trip Duration
Regression: Tabular data

### A Practical Guide to NY Taxi Data
(원본 커널) https://www.kaggle.com/onlyshadow/a-practical-guide-to-ny-taxi-data-0-379 <br>
(참고 한글 커널) https://right1203.github.io/study/2018/01/31/new-york-city-taxi-trip/ <br>

### Problem and Data Explanation
대회목적 : 뉴욕에서의 택시 여행 기간을 예측하는 모델을 만드는것

### Data fields
* id - a unique identifier for each trip
* vendor_id - a code indicating the provider associated with the trip record
* pickup_datetime - date and time when the meter was engaged
* dropoff_datetime - date and time when the meter was disengaged
* passenger_count - the number of passengers in the vehicle (driver entered value)
* pickup_longitude - the longitude where the meter was engaged
* pickup_latitude - the latitude where the meter was engaged
* dropoff_longitude - the longitude where the meter was disengaged
* dropoff_latitude - the latitude where the meter was disengaged
* store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
* trip_duration - duration of the trip in seconds


-------------------------------------
### what I learned
* 날짜 데이터를 csv파일 불러올때부터 date time으로 인식하게 불러오기
    * pd.read_csv(,parse_dates['컬럼 이름'])
    * to_datetime으로 변경해줘도 됨.
* datetime.strptime
    * 날짜를 나타내는 문자열을 반환. (ex. January 01 2016 을 2016,1,1 로 변환)
* map함수와 lambda함수
    * lambda함수는 익명함수이기 때문에 한번 쓰이고 다음줄로 넘어가는 힙 메모리 영역에서 증발
    * map함수는 함수에 의해 수행된 결과를 묶어서 map iterator객체로 리턴
        * 즉, map (f, iterable) 로 반복 가능한 iterable자료형이 있어야 한다.
* 자료형 series의 특징
    * Series의 특징은 값과 함께, 우리가 원하는 index를 입력할 수 있다.
* 위도,경도 데이터로 Distance를 구하는 여러 방법
    * Haversine distance
    * Manhattan distance
    * Bearing

---------------------------------
## 0. Import Modules and Data

In [12]:
#from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize']=(10,18)
%matplotlib inline
from datetime import datetime
from datetime import date
import xgboost as xgb
from sklearn.cluster import MiniBatchKMeans
import seaborn as sns # plot beautiful charts
import warnings
sns.set()
warnings.filterwarnings('ignore')

In [13]:
data =pd.read_csv('./train.csv', parse_dates=['pickup_datetime'])
test=pd.read_csv('./test.csv', parse_dates=['pickup_datetime'])

In [14]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null datetime64[ns]
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 122.4+ MB


In [30]:
for df in (data, test):
    df['year']=df['pickup_datetime'].dt.year
    df['month']=df['pickup_datetime'].dt.month
    df['day']=df['pickup_datetime'].dt.day
    df['hr']=df['pickup_datetime'].dt.hour
    df['minute']=df['pickup_datetime'].dt.minute
    df['store_and_fwd_flag']=1*(df.store_and_fwd_flag.values=='Y')   # Y이면 1, N이면 0으로 변환

In [31]:
test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,year,month,day,hr,minute
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,0,2016,6,30,23,59
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,0,2016,6,30,23,59
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,0,2016,6,30,23,59
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,0,2016,6,30,23,59
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,0,2016,6,30,23,59


우리는 RMSLE를 우리 score의 metric으로 사용할것이기 때문에, trip_durationd을 log형태로 변환하자. <br>
trip_duration에 outlier가 있으면, 이 변환이 효과를 완화하는데 도움을 줄것이다. <br>
만약 log변환을 하고 나서도 value가 여전히 과도하게 크다면, 우리는 이것이 실제 outlier라고 보다 확실하게 말할 수 있다.

## 1. Features
가장 중요하다고 생각되는 features 는 <br>
1. the pickup time 
2. the trip distance
3. the pickup location

관련없는 정보를 고려하지 않아도 되는, Gradient Boost Tree를 사용할 것이다. <br>
또한 다중공선성은 고려하지 않아도 된다. 그래서 우리는 가능한 많은 feature를 넣을 것이다. <br>
Gradient Boost의 다른 좋은 점은, tree-based 알고리즘이기 때문에 missing values를 신경쓰지 않아도 된다는 점이다.

### 1.1 Pickup Time and Weekend Features
rush hour에는 travel time이 더 길것이고, work day에는 다른 결과를 가져올것이라 생각할 수 있다. <br>
우리는 pickup_datetime으로부터 아래의 feature들을 만들어낼것이다. <br>

1. rest_day : rest day일 경우 True, 아닐경우 False.
2. weekend : weekend일 경우 True, 아닐경우 False.
3. pickup_time : 7:30 am 이면 7.5, 6:45pm 이면 18.75

In [47]:
from datetime import datetime
holiday=pd.read_csv('./NYC_2016Holidays.csv',sep=';')

In [49]:
holiday['Date'] = holiday['Date'].apply(lambda x: x + ' 2016')

In [54]:
holiday.head()

,Day,Date,Holiday
0,Friday,January 01 2016,New Years Day
1,Monday,January 18 2016,Martin Luther King Jr. Day
2,Friday,February 12 2016,Lincoln's Birthday
3,Monday,February 15 2016,Presidents' Day
4,Sunday,May 08 2016,Mother's Day


In [71]:
holidays = [datetime.strptime(holiday.loc[i,'Date'], '%B %d %Y').date() for i in range(len(holiday))]
holidays

[datetime.date(2016, 1, 1),
 datetime.date(2016, 1, 18),
 datetime.date(2016, 2, 12),
 datetime.date(2016, 2, 15),
 datetime.date(2016, 5, 8),
 datetime.date(2016, 5, 30),
 datetime.date(2016, 6, 19),
 datetime.date(2016, 7, 4),
 datetime.date(2016, 9, 5),
 datetime.date(2016, 10, 10),
 datetime.date(2016, 11, 11),
 datetime.date(2016, 11, 24),
 datetime.date(2016, 12, 26),
 datetime.date(2016, 7, 4),
 datetime.date(2016, 11, 8)]

In [59]:
time_data = pd.DataFrame(index = range(len(data)))
time_test = pd.DataFrame(index = range(len(test)))

In [73]:
from datetime import date
def restday(yr,month,day,holidays):
    '''
    Output:
        is_rest : sample이 rest day에 발생했을 경우의 Boolean variable
        is_weekend: sample 이 weekend에 발생했을 경우의 Boolean variable
    '''
    is_rest = [None]*len(yr)     # None값을 year의 길이만큼 대입
    is_weekend=[None]*len(yr)    
    i=0
    for yy,mm,dd in zip(yr,month,day):
        is_weekend[i]=date(yy,mm,dd).isoweekday() in (6,7)   # isoweekday()은 정수로 요일을 반환. 월요일은 1, 일요일은 7
        is_rest[i] = is_weekend[i] or date(yy,mm,dd) in holidays
        i+=1
    return is_rest, is_weekend

In [78]:
# train data
rest_day,weekend = restday(data.year,data.month,data.day,holidays)
time_data = time_data.assign(rest_day=rest_day)
time_data = time_data.assign(weekend=weekend)
### 위의 assign함수로 하는것과 아래는 같은 결과
#time_data['rest_day']=rest_day
#time_data['weekend']=weekend
#time_data['pickup_time']=data.hr+data.minute/60

# test data
rest_day,weekend = restday(test.year,test.month,test.day,holidays)
time_test = time_test.assign(rest_day=rest_day)
time_test = time_test.assign(weekend=weekend)

In [81]:
time_data=time_data.assign(pickup_time= data.hr+data.minute/60)
time_test=time_test.assign(pickup_time=test.hr+test.minute/60)

In [83]:
time_data.to_csv('./time_data.csv', index=False)
time_test.to_csv('./time_test.csv', index=False)

In [84]:
time_data.head()

,rest_day,weekend,pickup_time
0,False,False,17.400000
1,True,True,0.716667
2,False,False,11.583333
3,False,False,19.533333
4,True,True,13.500000


### 1.2 Distance Features


#### 1.2.1 OSRM Features
GPS로부터 얻은 실제 pickup과 dropoff의 위치 차이가 아니라, travel distance가 더 관련성 있는 데이터라고 한다.

In [85]:
# csv파일 불러올때부터 특정 컬럼만 불러오기
fastrout1 = pd.read_csv('./fastest_routes_train_part_1.csv', usecols=['id','total_distance','total_travel_time','number_of_steps','step_direction'])
fastrout2 = pd.read_csv('./fastest_routes_train_part_2.csv', usecols=['id','total_distance','total_travel_time','number_of_steps','step_direction'])
fastrout = pd.concat((fastrout1, fastrout2))
fastrout.head()

,id,total_distance,total_travel_time,number_of_steps,step_direction
0,id2875421,2009.1,164.9,5,left|straight|right|straight|arrive
1,id2377394,2513.2,332.0,6,none|right|left|right|left|arrive
2,id3504673,1779.4,235.8,4,left|left|right|arrive
3,id2181028,1614.9,140.1,5,right|left|right|left|arrive
4,id0801584,1393.5,189.4,5,right|right|right|left|arrive


right turns와 left turns의 수에 대한 feature을 만들것이다. 'slight right'와 'slight left'는 카운트하지 않을 것이다.

In [92]:
# map함수는 데이터 각각에 특정 함수를 적용.
### map(f, iterable) : 함수 (f)와 반복 가능한 (iterable)자료형을 입력으로 받는다.
# lambda를 통해 즉석에서 함수를 만들어서 적용.
right_turn=[]
left_turn=[]
right_turn+= list(map(lambda x:x.count('right')-x.count('slight right'), fastrout.step_direction))   # 여기서 iterable자료형은 fastrout.step_direction값
left_turn += list(map(lambda x:x.count('left')-x.count('slight left'),fastrout.step_direction))

trip_duration과 관련있는 feature들을 결합해보자.

In [105]:
osrm_data=fastrout[['id','total_distance','total_travel_time','number_of_steps']]
osrm_data['right_steps']=right_turn
osrm_data['left_steps']=left_turn
osrm_data.head()

,id,total_distance,total_travel_time,number_of_steps,right_steps,left_steps
0,id2875421,2009.1,164.9,5,1,1
1,id2377394,2513.2,332.0,6,2,2
2,id3504673,1779.4,235.8,4,1,2
3,id2181028,1614.9,140.1,5,2,2
4,id0801584,1393.5,189.4,5,3,1


OSRM 데이터의 행은 1458643개이며, 실제 데이터보다 1개 행이적다. 그래서 이 데이터를 사용하기 위해서는 SQL의 join을 사용하여서 데이터를 접합시켜야 한다.

In [109]:
len(osrm_data)

1458643

In [107]:
len(data)

1458644

In [114]:
data= data.join(osrm_data.set_index('id'), on='id')
data.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,year,month,day,hr,minute,total_distance,total_travel_time,number_of_steps,right_steps,left_steps
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,...,2016,3,14,17,24,2009.1,164.9,5.0,1.0,1.0
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,...,2016,6,12,0,43,2513.2,332.0,6.0,2.0,2.0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,...,2016,1,19,11,35,11060.8,767.6,16.0,5.0,4.0


테스트 데이터에도 동일한 처리를 한다.

In [133]:
osrm_test=pd.read_csv('./fastest_routes_test.csv')
right_turn= list(map(lambda x:x.count('right')-x.count('slight right'),osrm_test.step_direction))
left_turn = list(map(lambda x:x.count('left')-x.count('slight left'),osrm_test.step_direction))

osrm_test = osrm_test[['id','total_distance','total_travel_time','number_of_steps']]
osrm_test = osrm_test.assign(right_steps=right_turn)
osrm_test = osrm_test.assign(left_steps=left_turn)
osrm_test.head(3)

,id,total_distance,total_travel_time,number_of_steps,right_steps,left_steps
0,id0771704,1497.1,200.2,7,2,3
1,id3274209,1427.1,141.5,2,0,0
2,id2756455,2312.3,324.6,9,4,4


In [135]:
len(test)

625134

In [136]:
len(osrm_test)

625134

In [ ]:
test = test.join(osrm_test.set_index('id'), on='id')

In [138]:
osrm_data = data[['total_distance','total_travel_time','number_of_steps','right_steps','left_steps']]
osrm_test = test[['total_distance','total_travel_time','number_of_steps','right_steps','left_steps']]

In [139]:
data.to_csv('./osrm_data.csv',index=False, columns=['total_distance','total_travel_time','number_of_steps','right_steps','left_steps'])
test.to_csv('./osrm_test.csv',index=False,
            columns = ['total_distance','total_travel_time','number_of_steps','right_steps','left_steps'])

#### 1.2.2 Other Distance Features
1. Haversine distance: the direct distance of two GPS location, taking into account that the earth is round.
2. Manhattan distance: the usual L1 distance, here the haversine distance is used to calculate each coordinate of distance.
3. Bearing: The direction of the trip. Using radian as unit. (I must admit that I am not fully understand the formula. I have starring at it for a long time but can't come up anything. If anyone can help explain that will do me a big favor.)

- 출처 : beluga

In [147]:
def haversine_array(lat1,lng1,lat2,lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))   # dropoff위/경도, pickup위/경도를 np.radians함수로 변경
    AVG_EARTH_RADIUS=6371
    lat= lat2 - lat1
    lng= lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
    a = haversine_array(lat1, lng1, lat1, lng2)
    b = haversine_array(lat1, lng1, lat2, lng1)
    return a + b

def bearing_array(lat1, lng1, lat2, lng2):
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [161]:
List_dist = []
for df in (data,test):
    lat1, lng1, lat2, lng2 = (df['pickup_latitude'].values, df['pickup_longitude'].values, 
                              df['dropoff_latitude'].values,df['dropoff_longitude'].values)
    dist = pd.DataFrame(index=range(len(df)))
    dist = dist.assign(haversind_dist = haversine_array(lat1, lng1, lat2, lng2))
    dist = dist.assign(manhattan_dist = dummy_manhattan_distance(lat1, lng1, lat2, lng2))
    dist = dist.assign(bearing = bearing_array(lat1, lng1, lat2, lng2))
    List_dist.append(dist)   # List_dist에는 train데이터와 test데이터 각각이 들어가있음.
Other_dist_data,Other_dist_test = List_dist   # 각각에 train데이터와 test데이터

In [165]:
Other_dist_data.to_csv('./Other_dist_data.csv', index=False)
Other_dist_test.to_csv('./Other_dist_test.csv',index=False)

### 1.3 Location Features: K-means Clustering
location의 town별로 points를 나누려고 한다. 이것은 google API 또는 OSRM로부터의 몇가지 지리학적 정보를 요구할것이다. <br>
우리 데이터셋에 10개의 kmeans cluster를 셋팅할것이다. <br>
kmeans 는 4-d data를 수행한다. ['pickup_latitude', 'pickup_longitude','dropoff_latitude', 'dropoff_longitude'] <br>

결과 feature는 0,1,2,,,10사이의 categorical value이다. 이 feature를 xgboost에서 쓰려면, 20dummy variables로 변환해야한다. <br>
그렇지 않으면, 모듈은 이것을 numerical로 다룰것이다.